In [7]:
import numpy as np, matplotlib.pyplot as plt, random, json, pickle, datetime, copy, socket, math
from scipy.stats import sem
import matplotlib.colors as colors
from scipy.ndimage import gaussian_filter as gauss # for smoothing ratemaps
import sys, difflib
from importlib import reload

from numpy.linalg import norm as npNorm

if socket.gethostname() == 'Tolman':
    codeDirBase = 'C:\\Users\\whockei1\\Google Drive'
elif socket.gethostname() == 'DESKTOP-BECTOJ9':
    codeDirBase = 'C:\\Users\\whock\\Google Drive'
    
sys.path.insert(0, codeDirBase + '\\KnierimLab\\Ratterdam\\Code')
sys.path.insert(0, codeDirBase + '\\Python_Code\\KLab\\mts_analysis')
import utility_fx as util
import ratterdam_ParseBehavior as pBehav
from ratterdam_Defaults import *
import ratterdam_CoreDataStructures as core
import ratterdam_Directionality as Dir


%qtconsole --style native

In [139]:
#s = {i:stimuli['stimuli'][i+1] for i in range(17)}
#o = [(p[0],p[1]-1) for p in stimuli['order']]
#stimuli2 = {'stimuli':s,'order':o}
#pickle.dump(stimuli2,open(datafile+stimFile,"wb"))

In [2]:
exp = 'RFD3'
datafile = f'E:\\Ratterdam\\R765\\R765{exp}\\'
stimFile = util.getStimFileName(datafile)
stimuli = pickle.load(open(datafile+stimFile,'rb'))
print(stimuli)

{'stimuli': {0: [array(['A'],
      dtype='<U1'), array(['C'],
      dtype='<U1'), array(['B'],
      dtype='<U1'), array(['C'],
      dtype='<U1'), array(['B'],
      dtype='<U1'), array(['C'],
      dtype='<U1')], 1: [array(['C'],
      dtype='<U1'), array(['B'],
      dtype='<U1'), array(['A'],
      dtype='<U1'), array(['B'],
      dtype='<U1'), array(['A'],
      dtype='<U1'), array(['B'],
      dtype='<U1'), array(['A'],
      dtype='<U1')], 2: [array(['C'],
      dtype='<U1'), array(['A'],
      dtype='<U1'), array(['B'],
      dtype='<U1'), array(['C'],
      dtype='<U1')], 3: [array(['A'],
      dtype='<U1'), array(['B'],
      dtype='<U1'), array(['A'],
      dtype='<U1'), array(['C'],
      dtype='<U1'), array(['B'],
      dtype='<U1')], 4: [array(['B'],
      dtype='<U1'), array(['A'],
      dtype='<U1'), array(['B'],
      dtype='<U1'), array(['C'],
      dtype='<U1'), array(['A'],
      dtype='<U1'), array(['B'],
      dtype='<U1'), array(['A'],
      dtype='<U1'), array(

In [18]:
reload(core)
reload(pBehav)
behav = core.BehavioralData(datafile,exp, 0)
ts, position, alleyTracking, alleyVisits,  txtVisits = behav.loadData()

2087915.21235
2093265.15151


In [19]:
unit = core.UnitData("TT10\\cl-maze1.1", datafile, exp, alleyBounds, alleyVisits, txtVisits, position, ts)
unit.loadData_raw()

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\Ratterdam\\R765\\R765RFD3\\TT10\\cl-maze1.1'

In [21]:
def extractCorners(givenAlleyBounds):
    """Alley bounds gives [[x1, x2], [y1, y2]]. Convert that
    to UL, LL, UR, LL (xn, ym) points n,m <- [1,2]
    ul - x1, y2
    ll - x1, y1
    ur = x2, y2
    lr = x2, y1
    
    Returns ul, ll, ur, lr
    """
    b = givenAlleyBounds # for ease of typing
    ul, ll, ur, lr = [b[0][0], b[1][1]], [b[0][0], b[1][0]], [b[0][1], b[1][1]], [b[0][1], b[1][0]]
    return ul, ll, ur, lr

def checkCloserPoint(p1, p2, pt):
    """
    Given two points p1, p2
    where each is [x,y]
    see which pt is closer to
    (also of form [x,y])
    
    Return string "first" or "second"
    meaning its closer to first point arg
    or second point arg. If equal return "error"
    """
    d1 = npNorm(p1 - pt)
    d2 = npNorm(p2 - pt)
    if d1 < d2:
        return "first"
    elif d2 < d1:
        return "second"
    else:
        return None
    
    
def checkVisitEntrySide(visitOccs, bounds):
    """
    visitOccs is [ts,x,y] arr for 1 visit
    bounds is [ul, ll, ur, lr] for alley in question
    Return a label "SW" or "NE"
    """
    begin = visitOccs[0,1:]
    ll, ur = bounds[1], bounds[2]
    closerPt = checkCloserPoint(ll, ur, begin)
    if closerPt is not None:
        if closerPt == "first":
            side = "SW"
        elif closerPt == "second":
            side = "NE"
    else:
        side = None
    return side


def groupVisitsByDir(alley):
    """
    Given a 1-idx alley, consider all
    visits and group them by whether entry
    was from SW or NE side.
    
    Method checks if 1st occ pt is closer
    to LL or UR corner of alley
    
    Returns dict list of visits from SW, NE
    """
    bounds = extractCorners(alleyBounds[alley-1]) # ul, ll, ur, lr format list
    visitDirs = {"SW":[], "NE":[]}
    for i,visit in enumerate(unit.alleys[alley]):
        side = checkVisitEntrySide(visit['occs'], bounds)
        if side is not None:
            visitDirs[side].append(i)
    
    return visitDirs

def getTxtVisitsByDir(alley):
    """
    Given an alley, group visits
    whether SW/NE entry and crossref
    with txts present to get table
    of txts by dir count
    """
    txtDirTable = {txt:{"SW":[], "NE":[]} for txt in ["A", "B", "C"]}
    visitDirs = groupVisitsByDir(alley)
    for side in ["SW", "NE"]:
        if visitDirs[side] is not []:
            for visitIdx in visitDirs[side]:
                txt = unit.alleys[alley][visitIdx]['metadata']['stimulus'][0] # 0 bc it has dtype as entry 1
                txtDirTable[txt][side].append(visitIdx)
    return txtDirTable

In [22]:
for i in range(1,18):
    table = getTxtVisitsByDir(i)
    print(f"Alley {i}")
    print(table)

Alley 1
{'A': {'SW': [], 'NE': []}, 'B': {'SW': [], 'NE': []}, 'C': {'SW': [], 'NE': []}}
Alley 2
{'A': {'SW': [], 'NE': []}, 'B': {'SW': [], 'NE': []}, 'C': {'SW': [], 'NE': []}}
Alley 3
{'A': {'SW': [], 'NE': []}, 'B': {'SW': [], 'NE': []}, 'C': {'SW': [], 'NE': []}}
Alley 4
{'A': {'SW': [], 'NE': []}, 'B': {'SW': [], 'NE': []}, 'C': {'SW': [], 'NE': []}}
Alley 5
{'A': {'SW': [], 'NE': []}, 'B': {'SW': [], 'NE': []}, 'C': {'SW': [], 'NE': []}}
Alley 6
{'A': {'SW': [], 'NE': []}, 'B': {'SW': [], 'NE': []}, 'C': {'SW': [], 'NE': []}}
Alley 7
{'A': {'SW': [], 'NE': []}, 'B': {'SW': [], 'NE': []}, 'C': {'SW': [], 'NE': []}}
Alley 8
{'A': {'SW': [], 'NE': []}, 'B': {'SW': [], 'NE': []}, 'C': {'SW': [], 'NE': []}}
Alley 9
{'A': {'SW': [], 'NE': []}, 'B': {'SW': [], 'NE': []}, 'C': {'SW': [], 'NE': []}}
Alley 10
{'A': {'SW': [], 'NE': []}, 'B': {'SW': [], 'NE': []}, 'C': {'SW': [], 'NE': []}}
Alley 11
{'A': {'SW': [], 'NE': []}, 'B': {'SW': [], 'NE': []}, 'C': {'SW': [], 'NE': []}}
Alley 12